## 5.4. 단순한 계층 구현하기

In [17]:
class MulLayer():
    def __init__(self):
        self.x = None
        self.y = None
    
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, dout):
        x = self.x
        y = self.y
        
        dx = y * dout
        dy = x * dout
        
        return dx, dy

In [18]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

## 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print(price)

## 역전파
d_apple_price, d_tax = mul_tax_layer.backward(dprice := 1)
d_apple, d_apple_num = mul_apple_layer.backward(d_apple_price)
print(d_apple, d_apple_num, d_tax)

220.00000000000003
2.2 110.00000000000001 200


In [19]:
class AddLayer():
    def __init__(self):
        pass
        
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
       dx = dy = dout
       
       return dx, dy

In [20]:
class Relu():
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
        
    
    def backward(self, dout):
        dx = dout.copy()
        dx[self.mask] = 0
        return dx

In [21]:
import numpy as np
layer = Relu()

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
layer.forward(x)

array([[1., 0.],
       [0., 3.]])

### 5.5.2. 시그모이드 계층

#### 직접 계산해보기

In [27]:
class DivLayer():
    def __init__(self):
        self.x = None
        
    def forward(self, x):
        self.x = x
        out = 1/x
        return out
    
    def backward(self, dout):
        x = self.x
        dx = -1/(x**2) * dout
        return dx


class ExpLayer():
    def __init__(self):
        self.x = None
    
    def forward(self, x):
        self.x = x
        out = np.exp(x)
        return out
    
    def backward(self, dout):
        x = self.x
        dx = np.exp(x) * dout
        return dx

In [ ]:
## 순전파
mul = MulLayer()
exp = ExpLayer()
add = AddLayer()
div = DivLayer()

x = np.array([1,2])
mul_n = -1
add_n = 1

mul_out = mul.forward(x, mul_n)
print(mul_out)

exp_out = exp.forward(mul_out)
print(exp_out)

add_out = add.forward(exp_out, add_n)
print(add_out)

div_out = div.forward(add_out)
print(div_out)

[-1 -2]
[0.36787944 0.13533528]
[1.36787944 1.13533528]
[0.73105858 0.88079708]


In [ ]:
## 역전파
d_div_out = div.backward(1)
print(d_div_out)

d_add_out, _ = add.backward(d_div_out)
print(d_add_out)

d_exp_out = exp.backward(d_add_out)
print(d_exp_out)

d_mul_out, _ = mul.backward(d_exp_out)
print(d_mul_out)

[-0.53444665 -0.77580349]
[-0.53444665 -0.77580349]
[-0.19661193 -0.10499359]
[0.19661193 0.10499359]


#### 시그모이드 모듈로 계산하기

In [28]:
class Sigmoid():
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        out = self.out
        dx = dout * out * (1 - out)
        return dx

In [41]:
sig = Sigmoid()

out = sig.forward(x)
print(out)

dx = sig.backward(1)
print(dx)

[0.73105858 0.88079708]
[0.19661193 0.10499359]


## 5.6. Affine/Softmax층 계산